## Import libraries

In [23]:
import pandas as pd
import re
import pickle

## Read The file using pandas:

In [2]:
df=pd.read_csv('ar_reviews_100k.tsv',sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


## Data analysis:

In [3]:
df.shape

(99999, 2)

###### The data has 99999 rows and 2 clumns:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   99999 non-null  object
 1   text    99999 non-null  object
dtypes: object(2)
memory usage: 781.3+ KB


### To check the missing data :

In [5]:
df.isna().sum()

label    0
text     0
dtype: int64

###### There is no missing data.

In [6]:
df['label'].value_counts()

Mixed       33333
Positive    33333
Negative    33333
Name: label, dtype: int64

## work on data :

In [7]:
df["label"]=df["label"].apply(lambda x:0 if x=='Mixed'  else(1 if x=="Positive"  else -1))

To convert data(label) from objects to integers.

In [8]:
df['label'].value_counts()

-1    33333
 1    33333
 0    33333
Name: label, dtype: int64

## Load stopwords 

In [9]:
stopwords = open("stopwords_ar.txt",'r',encoding='utf-8').read().split('\n') ## 1

In [10]:
stopwords

['،',
 'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آل',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آهاً',
 'آهٍ',
 'آهِ',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أكتوبر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنه',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلى',
 'إلي',
 'إليكم',
 'إليكما',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',
 'إمّا',
 'إن',
 'إنَّ',
 'إى',
 'إياك',
 'إياكم',
 'إياكما',
 'إياكن',


In [11]:
data = df[['text','label']]
data

,text,label
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1
...,...,...
99994,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...,-1
99995,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...,-1
99996,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...,-1
99997,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...,-1


## Text Pre-Processing
    Stop-words removal

In [12]:
def remove_stop_words(Text):
    filtered_words = []
    Text=re.sub('[^ء-ي]',' ',Text).replace("  ",' ')
    words = Text.split()
    for word in words:
        if word not in stopwords:
            filtered_words.append(word)
            
    return ' '.join(filtered_words)

In [13]:
remove_stop_words(data.iloc[0,0])

'ممتاز نوعا النظافة والموقع والتجهيز والشاطيء المطعم'

In [14]:
data['text_no_sw'] = data.text.apply(remove_stop_words)
data.head()

C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,label,text_no_sw
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,1,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,1,هادفة وقوية تنقلك صخب شوارع القاهرة هدوء جبال ...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,1,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,1,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...


In [15]:
data_1=data[['text_no_sw','label']]
data_1.head()

,text_no_sw,label
0,ممتاز نوعا النظافة والموقع والتجهيز والشاطيء ا...,1
1,أسباب نجاح الإمارات شخص الدولة يعشق ترابها نحب...,1
2,هادفة وقوية تنقلك صخب شوارع القاهرة هدوء جبال ...,1
3,خلصنا مبدئيا اللي مستني ابهار زي الفيل الازرق ...,1
4,ياسات جلوريا جزء يتجزأ دبي فندق متكامل الخدمات...,1


### To remove the empty Rows :

In [16]:
data_1.drop(data_1[data_1["text_no_sw"]==""].index,inplace=True)
data_1[data_1["text_no_sw"]==""]

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text_no_sw,label


#### To split data into Test and Train

In [17]:
x=data_1["text_no_sw"]
y=data_1["label"]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=90)

## Applying machine algorithm 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [19]:
from sklearn.pipeline import Pipeline
pipeline_lr = Pipeline([
                      ('tf',TfidfVectorizer())
                     ,('lr_classifier',LogisticRegression())])

In [20]:
pipeline_lr.fit(x_train, y_train)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('tf', TfidfVectorizer()),
                ('lr_classifier', LogisticRegression())])

In [21]:
pipeline_lr.score(x_train,y_train)

0.8522936123926335

###### by using LogisticRegression that give us more accuracy ,and i used pipelines to be faster .  

## to save the model using pickle: 

In [24]:
pickle.dump(pipeline_lr,open('reviews.pkl', 'wb'))

In [25]:
model=pickle.load(open('reviews.pkl', 'rb'))

## The aim of the project :

##### The Target of this project is create machine algorithm that take The review sentences and decide if its positive or negative or mixed that will help us to know the general reviews status of the visitors. 

# The summry 

>+ At the first i imported the libraries and  read the file using pandas <br>
>+ The data has 99999 rows and 2 clumns and There is no missing data. <br>
>+ I converted the data(label) from objects to integers use it in machine algorithm after .<br>
>+ I removed the stopwords and empty rows<br>
>+ Splited the data into Test and Train and i applied LogisticRegression because it  gave us the highest accuracy ,and i used pipelines to be faster .<br>
>+ Then i debloyed the algorithm <br>
>+ Then i created algorithm to save the new review sentiment with it new label statement in review record.csv <br>